In [3]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split,StratifiedKFold # Model evaluation
from sklearn.preprocessing import LabelEncoder, RobustScaler, OneHotEncoder, StandardScaler # Preprocessing
from sklearn.linear_model import Lasso, Ridge, ElasticNet,  LassoLarsIC, RANSACRegressor, SGDRegressor, HuberRegressor, BayesianRidge # Linear models
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor  # Ensemble methods
from xgboost import XGBRegressor, plot_importance # XGBoost
from sklearn.svm import SVR, SVC, LinearSVC  # Support Vector Regression
from sklearn.tree import DecisionTreeRegressor # Decision Tree Regression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.pipeline import Pipeline, make_pipeline # Streaming pipelines
from sklearn.decomposition import KernelPCA, PCA # Dimensionality reduction
from sklearn.feature_selection import SelectFromModel # Dimensionality reduction
from sklearn.model_selection import learning_curve, validation_curve, GridSearchCV # Model evaluation
from sklearn.base import clone, BaseEstimator, TransformerMixin, RegressorMixin # Clone estimator
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import explained_variance_score, f1_score, roc_auc_score, median_absolute_error, r2_score, mean_squared_error #To evaluate our model
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, fbeta_score #To evaluate our model
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split 
import optuna

In [4]:
import pandas as pd
import numpy as np
import re
import numpy as np

In [5]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
testId = test.Property_Id
train.Price_Category = train.Price_Category.replace("Cheap",0).replace("Affordable",1).replace("Semi-Premium",2).replace("Premium",3)
y = train.Price_Category
train = train.drop(["Property_Id", "Price_Category"],axis=1)
test = test.drop(["Property_Id"],axis=1)
features = pd.concat([train,test])
train.head(1)

,Title,Label,Type,Location,Baths,Area,Purpose,Bedroom(s),Description,Main Features,Rooms,Business and Communication,Healthcare Recreational,Nearby Locations and Other Facilities,Other Facilities,Popular,Source,Time Stamp
0,Brand New 1 Kanal House For Sale In Bahria Tow...,"Bahria Town - Gulbahar Block, Bahria Town - Se...",House,"Bahria Town, Lahore, Punjab",6,1 Kanal,For Sale,5,Brand New 1 Kanal House For Sale In Bahria Tow...,"Built in year: 2018 , Parking Spaces: 4 , Dou...","Bedrooms: 5 , Bathrooms: 7 , Servant Quarters...","Broadband Internet Access , Satellite or Cabl...","Lawn or Garden , Swimming Pool","Nearby Schools , Nearby Hospitals , Nearby Sh...","Maintenance Staff , Security Staff , Faciliti...",super hot,Lahore-1-10,3/31/2019 11:01


In [6]:
def findWcolon(a,x, length):
    val = re.findall("%s: \d+" % a,x)
    val = val[0][-length:] if len(val)>0 else (np.nan if len(x)==1 else 0)
    return val
def find(a,x):
    return 1 if a in x else (np.nan if len(x)==1 else 0)

def rooms(x):
    living = find("Study Room",x) + find("Drawing Room",x) + find("Dining Room",x) + find("Lounge or Sitting Room", x)
    athletic = find("Gym",x) + find("Steam Room",x) + find("Laundry Room",x) + find("Powder Room", x)
    #findWcolon("Bathrooms",x,1),
    return [findWcolon("Servant Quarters",x,1), findWcolon("Kitchens",x,1), findWcolon("Store Rooms",x,1), living, athletic, find("Other Rooms", x), find("Steam Room",x), find("Laundry Room",x), find("Powder Room", x), find("Study Room",x), find("Drawing Room",x), find("Dining Room",x), find("Lounge or Sitting Room", x)]

def main_features(x):
    return [findWcolon("Built in year",x,4), findWcolon("Parking Spaces",x,1), findWcolon("Floors",x,1), find("Double Glazed Windows", x), find("Flooring", x), find("Electricity Backup", x), find("Waste Disposal", x),find("Central Air Conditioning", x), find("Flooring", x)]

def business(x):
    return [find("Broadband Internet Access", x), find("Satellite or Cable TV Ready", x), find("Intercom", x), find("Other Business and Communication Facilities", x)]

def health(x):
    return [find("Lawn or Garden", x), find("Sauna", x) + find("Jacuzzi", x), find("Swimming Pool", x), find("Other Healthcare and Recreation Facilities", x)]

def facilities(x):
    cols = ['Maintenance Staff','Security Staff','Facilities for Disabled','Other Facilities']
    return [find(y,x) for y in cols]

def nearby(x):
    val = len([y for y in x.split(" , ") if "Nearby" in y]) if x != " " else np.nan
    return val


def process(train):
    train["nearbyVal"] = [1 if len(x) > 1 else 0 for x in train["Nearby Locations and Other Facilities"]]
    train["bussinessVal"] = [1 if len(x) > 1 else 0 for x in train["Business and Communication"]]
    train["staffVal"] = [1 if len(x) > 1 else 0 for x in train["Other Facilities"]]
    train["healthVal"] = [1 if len(x) > 1 else 0 for x in train["Healthcare Recreational"]]
    train[["Date","Parking","Floors","Windows", "Flooring", "ElectricBackup", "WasteDisposal", "CentralAir", "CentralHeating"]] = list(train["Main Features"].apply(main_features))
    train[["Servants","Kitchens","Store Rooms", "living","healthRooms","OtherRooms", "Steam Room","Laundry Room","Powder Room","Study Room","Drawing Room","Dining Room","Lounge"]] = list(train.Rooms.apply(rooms))
    train[["internet","tv","intercom","otherBusiness"]] = list(train["Business and Communication"].apply(business))
    train[["town","city","province"]] = list(train.Location.apply(lambda x: x.split(",")))
    train[["lawn","hotTub","pool","otherHealth"]] = list(train["Healthcare Recreational"].apply(health))
    train.Area = ["".join(x.split(",")) for x in train.Area]
    train.Area = train.Area.apply(lambda x: float(x.split(" ")[0]) if x.split(" ")[1] == "Marla" else float(x.split(" ")[0]) *20)
    train[['MaintenanceStaff','SecurityStaff','FacilitiesDisabled','ExtraFacilities']] = list(train["Other Facilities"].apply(facilities))
    train["nearby2"] = train["Nearby Locations and Other Facilities"].apply(nearby)
    train.Popular = train.Popular.replace("hot",1).replace("super hot",2)
    train = pd.concat([train,pd.get_dummies(train.Location)],axis=1)
    train[["Source", "SourceNum"]] = [["-".join(x.split("-")[:-1]), x.split("-")[-1] ]for x in train.Source]
    train = pd.concat([train,pd.get_dummies(train.Source)],axis=1)
    train = train.replace("-",0)
    train = train.drop(["Title","Label","Date", "city", "province", "town","Type","Purpose", "Healthcare Recreational", "Other Facilities", "Rooms","Nearby Locations and Other Facilities","Main Features", "Source", "Time Stamp", "Description", "Location", "Business and Communication"],axis=1)
    train = train.fillna(0)
    train = train.astype('int')
    return train

In [7]:
features = process(features)
train = features[:len(y)]
test = features[len(y):]
train = train.values
test = test.values


In [23]:
#Set to False if you want to skip it
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.33, random_state=42)
OPTUNA_OPTIMIZATION = True
N_SPLITS = 10 #Number of folds for validation
N_TRIALS = 100 #Number of trials to find best hyperparameters

In [24]:
def objective(trial, cv=StratifiedKFold(N_SPLITS, shuffle = True, random_state = 29)):
    
    
    param_lgb = {
        "random_state": trial.suggest_int("random_state", 1, 100),
        "objective": "multiclass",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "max_depth": trial.suggest_int("max_depth", -1, 10),
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
        "subsample": trial.suggest_float("subsample", 0.4, 1.0),
        "subsample_freq": trial.suggest_int("subsample_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    
    model = LGBMClassifier(**param_lgb)
    
    model.fit(X_train,y_train)
    predictions = model.predict(X_test).round(0)
    f1 = f1_score(y_test, predictions, average='macro')
    print("F1 %s" % (f1))
    return np.average(f1)

In [25]:
if OPTUNA_OPTIMIZATION:
    study = optuna.create_study(study_name = 'lgbm_parameter_opt', direction="maximize")
    study.optimize(objective, n_trials=N_TRIALS) 
    
    trial = study.best_trial
    
    print("  Value: {}".format(trial.value))
    
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
else:
    trial = {"reg_alpha": 0.362136938773081,
             "reg_lambda": 2.930297242488071,
             "max_depth": 10,
             "n_estimators": 306,
             "num_leaves": 71,
             "colsample_bytree": 0.7121396258381646,
             "subsample": 0.793959734582999,
             "subsample_freq": 2,
             "min_child_samples": 18}

[I 2021-03-22 22:07:05,461] A new study created in memory with name: lgbm_parameter_opt
[I 2021-03-22 22:07:15,362] Trial 0 finished with value: 0.6337102958795647 and parameters: {'random_state': 14, 'reg_alpha': 0.0005145454903214353, 'reg_lambda': 0.002061895356657936, 'max_depth': 2, 'n_estimators': 454, 'num_leaves': 203, 'colsample_bytree': 0.4724075604101913, 'subsample': 0.5768261645540214, 'subsample_freq': 6, 'min_child_samples': 61}. Best is trial 0 with value: 0.6337102958795647.


F1 0.6337102958795647


[I 2021-03-22 22:07:50,385] Trial 1 finished with value: 0.6538588636747774 and parameters: {'random_state': 49, 'reg_alpha': 0.06263438788489935, 'reg_lambda': 0.08181019944882222, 'max_depth': 8, 'n_estimators': 500, 'num_leaves': 141, 'colsample_bytree': 0.7936671498924196, 'subsample': 0.503692574076756, 'subsample_freq': 2, 'min_child_samples': 30}. Best is trial 1 with value: 0.6538588636747774.


F1 0.6538588636747774


[I 2021-03-22 22:08:06,427] Trial 2 finished with value: 0.6431155623333389 and parameters: {'random_state': 13, 'reg_alpha': 1.0985753284221587e-06, 'reg_lambda': 0.036570962237742186, 'max_depth': 0, 'n_estimators': 404, 'num_leaves': 14, 'colsample_bytree': 0.9651731988611744, 'subsample': 0.48587551814435714, 'subsample_freq': 1, 'min_child_samples': 64}. Best is trial 1 with value: 0.6538588636747774.


...

F1 0.7321939531695523


[I 2021-03-22 22:18:21,368] Trial 98 finished with value: 0.7442507652173438 and parameters: {'random_state': 53, 'reg_alpha': 0.22510529159798284, 'reg_lambda': 6.750492841274968e-07, 'max_depth': 3, 'n_estimators': 152, 'num_leaves': 135, 'colsample_bytree': 0.5481176869940257, 'subsample': 0.8011220054580684, 'subsample_freq': 1, 'min_child_samples': 7}. Best is trial 98 with value: 0.7442507652173438.


F1 0.7442507652173438


[I 2021-03-22 22:18:24,032] Trial 99 finished with value: 0.7453903078762071 and parameters: {'random_state': 53, 'reg_alpha': 0.22204450496336606, 'reg_lambda': 5.788896130674745e-07, 'max_depth': 3, 'n_estimators': 153, 'num_leaves': 131, 'colsample_bytree': 0.5452045748900861, 'subsample': 0.8016089617809282, 'subsample_freq': 1, 'min_child_samples': 7}. Best is trial 99 with value: 0.7453903078762071.


F1 0.7453903078762071
  Value: 0.7453903078762071
  Params: 
    random_state: 53
    reg_alpha: 0.22204450496336606
    reg_lambda: 5.788896130674745e-07
    max_depth: 3
    n_estimators: 153
    num_leaves: 131
    colsample_bytree: 0.5452045748900861
    subsample: 0.8016089617809282
    subsample_freq: 1
    min_child_samples: 7


In [26]:
if OPTUNA_OPTIMIZATION:
    final_model = LGBMRegressor(**trial.params)
else:
    final_model = LGBMRegressor(**trial)

In [27]:
trial.params

{'random_state': 53,
 'reg_alpha': 0.22204450496336606,
 'reg_lambda': 5.788896130674745e-07,
 'max_depth': 3,
 'n_estimators': 153,
 'num_leaves': 131,
 'colsample_bytree': 0.5452045748900861,
 'subsample': 0.8016089617809282,
 'subsample_freq': 1,
 'min_child_samples': 7}

In [37]:
model = LGBMClassifier(**trial.params)
    
model.fit(X_train,y_train)
predictions = model.predict(X_test).round(0)
f1 = f1_score(y_test, predictions, average='macro')
print("F1 %s" % (f1))
preds = model.predict(test).round(0)


F1 0.7453903078762071


In [33]:
def run_model(name, model,train,y,test):
    X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.33, random_state=42)
    model2 = model
    model2.fit(X_train,y_train)
    predictions = model.predict(X_test).round(0)
    print("%s F1 %s" % (name,f1_score(y_test, predictions, average='macro')))
    model.fit(train,y)
    preds = model.predict(test).round(0)
    return preds

In [34]:
models = [
    ("KNN",KNeighborsClassifier(n_neighbors=5)),
    ("LGBM",LGBMRegressor(**trial.params)),
    ("XGB",XGBRegressor())
]
for name,model in models:
    run_model(name, model, train,y,test)

UnboundLocalError: local variable 'X_train' referenced before assignment

In [18]:
name = models[1][0]
model = models[1][1]
preds = run_model(name, model, train,y,test)

LGBM F1 0.7340929540637331


In [38]:
preds = [["Cheap","Affordable","Semi-Premium","Premium"][int(x)] for x in preds.round(0)]

In [39]:
sub = pd.DataFrame({"Property_Id":testId,"Price_Category":preds})

In [40]:
sub.to_csv("../output/sub_optuna2.csv",index=False)

In [41]:
pre_sub = pd.read_csv("../output/sub_with_sourcenum.csv")
pre_sub.Price_Category.value_counts()/len(pre_sub)

Semi-Premium    0.600000
Premium         0.359633
Affordable      0.040367
Name: Price_Category, dtype: float64

In [42]:
pre_sub2 = pd.read_csv("../output/sub3.csv")
pre_sub2.Price_Category.value_counts()/len(pre_sub)

Semi-Premium    0.598165
Premium         0.357798
Affordable      0.042202
Cheap           0.001835
Name: Price_Category, dtype: float64

In [43]:
sub["Price_Category"].value_counts()

Semi-Premium    315
Premium         212
Affordable       18
Name: Price_Category, dtype: int64

In [44]:
pd.concat([sub,pre_sub],axis=1)[sub.Price_Category != pre_sub.Price_Category]

,Property_Id,Price_Category,Property_Id,Price_Category
36,1719,Premium,1719,Semi-Premium
52,711,Premium,711,Semi-Premium
87,1870,Semi-Premium,1870,Premium
100,278,Semi-Premium,278,Premium
116,3208,Premium,3208,Semi-Premium
124,739,Premium,739,Semi-Premium
141,973,Semi-Premium,973,Affordable
149,2185,Premium,2185,Semi-Premium
154,2029,Premium,2029,Semi-Premium
156,2103,Premium,2103,Semi-Premium


In [47]:
pd.concat([sub,pre_sub2],axis=1)[sub.Price_Category != pre_sub2.Price_Category]

,Property_Id,Price_Category,Property_Id,Price_Category
52,711,Semi-Premium,711,Premium
84,221,Semi-Premium,221,Affordable
176,3612,Affordable,3612,Semi-Premium
195,451,Semi-Premium,451,Premium
269,2576,Semi-Premium,2576,Premium
282,1586,Premium,1586,Semi-Premium
299,125,Semi-Premium,125,Affordable
303,2692,Premium,2692,Semi-Premium
368,1408,Semi-Premium,1408,Premium
377,2329,Affordable,2329,Semi-Premium
